<a href="https://colab.research.google.com/github/LuniaKunal/AI-powered-Mahabharata-Search/blob/main/MahaLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
with open('/content/combined_output.txt','r') as file:
    text = file.read()

In [ ]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  14080654


In [ ]:
# let's look at the first 1000 characters
print(text[:1000])

BOOK 1

THE MAHABHARATA

ADI PARVA

SECTION I

Om! Having bowed down to Narayana and Nara, the most exalted male being,
and also to the goddess Saraswati, must the word Jaya be uttered.

Ugrasrava, the son of Lomaharshana, surnamed Sauti, well-versed in the
Puranas, bending with humility, one day approached the great sages of
rigid vows, sitting at their ease, who had attended the twelve years'
sacrifice of Saunaka, surnamed Kulapati, in the forest of Naimisha. Those
ascetics, wishing to hear his wonderful narrations, presently began to
address him who had thus arrived at that recluse abode of the inhabitants
of the forest of Naimisha. Having been entertained with due respect by
those holy men, he saluted those Munis (sages) with joined palms, even
all of them, and inquired about the progress of their asceticism. Then
all the ascetics being again seated, the son of Lomaharshana humbly
occupied the seat that was assigned to him. Seeing that he was
comfortably seated, and recovered from 

In [ ]:
# Finding unique charaters present in the corpus
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f'Vocab size: {vocab_size}')
print(''.join(chars))

Vocab size: 79

 !"&'(),-.0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz


In [ ]:
# Creating a custom Encoder and Decoder
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[60, 61, 61, 1, 72, 60, 57, 70, 57]
hii there


In [ ]:
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([14080654]) torch.int64
tensor([25, 38, 38, 34,  1, 12,  0,  0, 43, 31, 28,  1, 36, 24, 31, 24, 25, 31,
        24, 41, 24, 43, 24,  0,  0, 24, 27, 32,  1, 39, 24, 41, 45, 24,  0,  0,
        42, 28, 26, 43, 32, 38, 37,  1, 32,  0,  0, 38, 65,  2,  1, 31, 53, 74,
        61, 66, 59,  1, 54, 67, 75, 57, 56,  1, 56, 67, 75, 66,  1, 72, 67,  1,
        37, 53, 70, 53, 77, 53, 66, 53,  1, 53, 66, 56,  1, 37, 53, 70, 53,  8,
         1, 72, 60, 57,  1, 65, 67, 71, 72,  1, 57, 76, 53, 64, 72, 57, 56,  1,
        65, 53, 64, 57,  1, 54, 57, 61, 66, 59,  8,  0, 53, 66, 56,  1, 53, 64,
        71, 67,  1, 72, 67,  1, 72, 60, 57,  1, 59, 67, 56, 56, 57, 71, 71,  1,
        42, 53, 70, 53, 71, 75, 53, 72, 61,  8,  1, 65, 73, 71, 72,  1, 72, 60,
        57,  1, 75, 67, 70, 56,  1, 33, 53, 77, 53,  1, 54, 57,  1, 73, 72, 72,
        57, 70, 57, 56, 10,  0,  0, 44, 59, 70, 53, 71, 70, 53, 74, 53,  8,  1,
        72, 60, 57,  1, 71, 67, 66,  1, 67, 58,  1, 35, 67, 65, 53, 60, 53, 70,
     

In [ ]:
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size+1]

tensor([25, 38, 38, 34,  1, 12,  0,  0, 43])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([25]) the target: 38
when input is tensor([25, 38]) the target: 38
when input is tensor([25, 38, 38]) the target: 34
when input is tensor([25, 38, 38, 34]) the target: 1
when input is tensor([25, 38, 38, 34,  1]) the target: 12
when input is tensor([25, 38, 38, 34,  1, 12]) the target: 0
when input is tensor([25, 38, 38, 34,  1, 12,  0]) the target: 0
when input is tensor([25, 38, 38, 34,  1, 12,  0,  0]) the target: 43


In [ ]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[64, 56,  1, 75, 61, 72, 60, 61],
        [72, 60, 57, 65,  8,  1,  3, 35],
        [59,  1, 71, 60, 67, 73, 64, 56],
        [66,  1, 72, 60, 67, 73,  1, 65]])
targets:
torch.Size([4, 8])
tensor([[56,  1, 75, 61, 72, 60, 61, 66],
        [60, 57, 65,  8,  1,  3, 35, 57],
        [ 1, 71, 60, 67, 73, 64, 56,  1],
        [ 1, 72, 60, 67, 73,  1, 65, 53]])
----
when input is [64] the target: 56
when input is [64, 56] the target: 1
when input is [64, 56, 1] the target: 75
when input is [64, 56, 1, 75] the target: 61
when input is [64, 56, 1, 75, 61] the target: 72
when input is [64, 56, 1, 75, 61, 72] the target: 60
when input is [64, 56, 1, 75, 61, 72, 60] the target: 61
when input is [64, 56, 1, 75, 61, 72, 60, 61] the target: 66
when input is [72] the target: 60
when input is [72, 60] the target: 57
when input is [72, 60, 57] the target: 65
when input is [72, 60, 57, 65] the target: 8
when input is [72, 60, 57, 65, 8] the target: 1
when input is [72,

In [ ]:
print(xb) # our input to the transformer

tensor([[64, 56,  1, 75, 61, 72, 60, 61],
        [72, 60, 57, 65,  8,  1,  3, 35],
        [59,  1, 71, 60, 67, 73, 64, 56],
        [66,  1, 72, 60, 67, 73,  1, 65]])


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 79])
tensor(4.8761, grad_fn=<NllLossBackward0>)

K'jbwY-AlaH[C_Jm:JBsJ.]PF)01I!e SLQ(VbHImy61.WuuceZA ]PNbyk,JP GNH(:ZUt6M9 G:wp"PjPL.6MH6EWwuz,JqU.W


In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


4.535962104797363


In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


ZMC-;
aQpSbHe_K(jx1Bpgqdf8XCEO?Y-fbLJ:Rxb8Yu(aM5Zx;RdwrK)i-
YJ]sM'c3w]_7CS  w?u(:tunWDxqN6Cc[L3Eua.'By6]qk2xm6CHc!
4:k[EB?un
Zq?UNByR ))ZKdg(-VmnVCN_dm_o'5ZED"ShjbDPWvykUSQKVo-'(!Mc383[6Ew0n-dfYV6MjQ;R;7((,Hqdr&]P-z101?&i6MCJl9]EEzAq9ZCTk_J4LSZK.Xd"mbqc75)iJ3wILW0c8a5DTabF8hrmCWS?a h?PZxA;Zj(Lgte7fqi1T1KL(x9BYhBkCD""cILispNUSQ8aEpojAPjYk(P G(v w0vl D7gkIyU(SBVl;:?tu!g83od79r]LAOuFnD"BvauDtWun?Vhh[R90eL(8XY'SZMEZI!Jq;e'BjU5H37zN6TS,26ClAUNaOz8kN6yun5UZx9B?)3:1Pqs5KvV
tu3O8&)k458rxZkWvAFeLWzZk)S8l


In [ ]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [ ]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [ ]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

False

In [ ]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [ ]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)

In [ ]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [ ]:
k.var()

tensor(1.0449)

In [ ]:
q.var()

tensor(1.0700)

In [ ]:
wei.var()

tensor(1.0918)

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [ ]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [ ]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('/content/combined_output.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.211535 M parameters
step 0: train loss 4.5733, val loss 4.5715
step 100: train loss 2.6272, val loss 2.6380
step 200: train loss 2.5105, val loss 2.5093
step 300: train loss 2.4159, val loss 2.4259
step 400: train loss 2.3317, val loss 2.3368
step 500: train loss 2.2597, val loss 2.2731
step 600: train loss 2.1979, val loss 2.2036
step 700: train loss 2.1482, val loss 2.1578
step 800: train loss 2.1035, val loss 2.1053
step 900: train loss 2.0476, val loss 2.0543
step 1000: train loss 2.0196, val loss 2.0379
step 1100: train loss 1.9816, val loss 1.9799
step 1200: train loss 1.9451, val loss 1.9466
step 1300: train loss 1.9169, val loss 1.9260
step 1400: train loss 1.9030, val loss 1.9003
step 1500: train loss 1.8712, val loss 1.8673
step 1600: train loss 1.8228, val loss 1.8520
step 1700: train loss 1.8188, val loss 1.8396
step 1800: train loss 1.7880, val loss 1.8086
step 1900: train loss 1.7758, val loss 1.7776
step 2000: train loss 1.7726, val loss 1.7778
step 2100: train loss 1.